In [1]:
import os
import ntpath
import numpy as np
from keras.models import model_from_json
from sklearn.metrics import mean_squared_error
import math
import contextlib
from utils.data_utils import getPaths, read_and_resize, preprocess, deprocess
from tqdm import tqdm
import random

# Define the path to the directory containing all datasets
datasets_dir = "D:/IITG/UW/DATASETS FOR COMPARISION"

# Define the names of the datasets
dataset_names = ["EUVP", "UIEB", "Degraded"]

# Function to select a random subset of images from a dataset
def select_random_subset(data_dir, num_samples):
    test_paths = list(getPaths(data_dir))
    return random.sample(test_paths, num_samples)

# Function to test a dataset and get MSE and PSNR results
def test_dataset(data_dir, dataset_name, num_samples):
    test_paths = select_random_subset(data_dir, num_samples)
    
    print(f"Testing {dataset_name} ({num_samples} images):")
    
    checkpoint_dir = 'models/gen_p/'
    model_name_by_epoch = "model_15320_"
    
    model_h5 = checkpoint_dir + model_name_by_epoch + ".h5"
    model_json = checkpoint_dir + model_name_by_epoch + ".json"
    assert (os.path.exists(model_h5) and os.path.exists(model_json))
    
    with open(model_json, "r") as json_file:
        loaded_model_json = json_file.read()
    funie_gan_generator = model_from_json(loaded_model_json)
    funie_gan_generator.load_weights(model_h5)
    
    all_mse_values = []
    all_psnr_values = []

    for img_path in tqdm(test_paths):
        inp_img = read_and_resize(img_path, (256, 256))
        im = preprocess(inp_img)
        im = np.expand_dims(im, axis=0)
        
        # Suppress the verbose output from the predict method
        with open(os.devnull, 'w') as fnull:
            with contextlib.redirect_stdout(fnull):
                gen = funie_gan_generator.predict(im)
        
        gen_img = deprocess(gen)[0]
        
        mse = mean_squared_error(inp_img.flatten(), gen_img.flatten())
        all_mse_values.append(mse)
        
        psnr = 10 * math.log10((255 ** 2) / mse)
        all_psnr_values.append(psnr)

    return all_mse_values, all_psnr_values

for dataset_name in dataset_names:
    dataset_path = os.path.join(datasets_dir, dataset_name)
    dataset_mse, dataset_psnr = test_dataset(dataset_path, dataset_name, 890)
    print(f"  {dataset_name} - MSE: {np.mean(dataset_mse):.4f}, PSNR: {np.mean(dataset_psnr):.4f}")


Testing EUVP (890 images):


100%|████████████████████████████████████████████████████████████████████████████████| 890/890 [02:02<00:00,  7.26it/s]


  EUVP - MSE: 571.2612, PSNR: 23.6921
Testing UIEB (890 images):


100%|████████████████████████████████████████████████████████████████████████████████| 890/890 [02:23<00:00,  6.21it/s]


  UIEB - MSE: 797.0392, PSNR: 21.4631
Testing Degraded (890 images):


100%|████████████████████████████████████████████████████████████████████████████████| 890/890 [04:26<00:00,  3.34it/s]

  Degraded - MSE: 239.7014, PSNR: 26.2760
